<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Settings</a></span></li><li><span><a href="#Prepare-dataframe-for-images-and-masks" data-toc-modified-id="Prepare-dataframe-for-images-and-masks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare dataframe for images and masks</a></span></li><li><span><a href="#Merge-ccRCC-and-pRCC" data-toc-modified-id="Merge-ccRCC-and-pRCC-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Merge ccRCC and pRCC</a></span></li></ul></div>

In [ ]:
import os
import pickle
import shutil
import pandas as pd

### Settings

In [ ]:
DATASET_TYPEs = ["ccRCC", "pRCC"]
FOLDs = ["train", "test"]
ROOT_PATH = "/space/ponzio/Morpheme_v2"
ROOT_PATH_DATA = os.path.join(ROOT_PATH, "data")
DF_DIR = "dataframes_vascular_segmentation_26-04-2021"

### Prepare dataframe for images and masks

In [ ]:
for DATASET_TYPE in DATASET_TYPEs:
    for FOLD in FOLDs:
        DIRPATH_DATA_SEGMENTATION = os.path.join(ROOT_PATH_DATA, "vascular_segmentation/{}/{}".format(FOLD, DATASET_TYPE))
        DIRPATH_DATASET_TYPE = os.path.join(ROOT_PATH_DATA, DATASET_TYPE)
        ROOT_PATH_DF = os.path.join(ROOT_PATH_DATA, DF_DIR, DATASET_TYPE, FOLD)
        if not os.path.exists(ROOT_PATH_DF):
            os.makedirs(ROOT_PATH_DF)
        else:
            print("%s exists yet!" %ROOT_PATH_DF)
        """
        Folder structure:
        Patient
            Annotation
        """
        filepath_images = list()
        filepath_masks = list()
        patients_train = list()
        for patient in os.listdir(DIRPATH_DATA_SEGMENTATION):
            patients_train += [patient]
            for annotation in os.listdir(os.path.join(DIRPATH_DATA_SEGMENTATION, patient)):
                filenames = os.listdir(os.path.join(DIRPATH_DATA_SEGMENTATION, patient, annotation))
                images = [f for f in filenames if f.endswith(".png") and "seg" not in f]
                masks = [f for f in filenames if f.endswith(".png") and "seg" in f]
                filepath_images += [os.path.join(DIRPATH_DATA_SEGMENTATION, patient, annotation, i) for i in images]
                filepath_masks += [os.path.join(DIRPATH_DATA_SEGMENTATION, patient, annotation, m) for m in masks]
        filepath_images.sort()
        filepath_masks.sort()
        images_df = pd.DataFrame(filepath_images, columns=["path"])
        masks_df = pd.DataFrame(filepath_masks, columns=["path"])
        images_df.to_csv(os.path.join(ROOT_PATH_DF, "images.csv"),index=False)
        masks_df.to_csv(os.path.join(ROOT_PATH_DF, "masks.csv"), index=False)
        print("{} - {}: found {} images".format(DATASET_TYPE, FOLD, len(images_df)))
        print("{} - {}: found {} masks".format(DATASET_TYPE, FOLD, len(masks_df)))

### Merge ccRCC and pRCC

In [ ]:
for FOLD in FOLDs:
    DIRPATH_DATA_SEGMENTATION_MERGED = os.path.join(ROOT_PATH_DATA, DF_DIR, "ccRCC+pRCC", FOLD)
    if not os.path.exists(DIRPATH_DATA_SEGMENTATION_MERGED):
        os.makedirs(DIRPATH_DATA_SEGMENTATION_MERGED)
    # ccRCC
    ROOT_PATH_DF_ccRCC = os.path.join(ROOT_PATH_DATA, DF_DIR, "ccRCC", FOLD)
    df_ccRCC_images = pd.read_csv(os.path.join(ROOT_PATH_DF_ccRCC, "images.csv"))
    df_ccRCC_masks = pd.read_csv(os.path.join(ROOT_PATH_DF_ccRCC, "masks.csv"))
    # pRCC
    ROOT_PATH_DF_pRCC = os.path.join(ROOT_PATH_DATA, DF_DIR, "pRCC", FOLD)
    df_pRCC_images = pd.read_csv(os.path.join(ROOT_PATH_DF_pRCC, "images.csv"))
    df_pRCC_masks = pd.read_csv(os.path.join(ROOT_PATH_DF_pRCC, "masks.csv"))
    # Merging
    df_images = pd.concat([df_ccRCC_images, df_pRCC_images])
    df_masks = pd.concat([df_ccRCC_masks, df_pRCC_masks])
    print(FOLD)
    print("ccRCC + pRCC: found {} images".format(len(df_images)))
    print("ccRCC + pRCC: found {} masks".format(len(df_masks)))
    df_images.to_csv(os.path.join(DIRPATH_DATA_SEGMENTATION_MERGED, "images.csv"),index=False)
    df_masks.to_csv(os.path.join(DIRPATH_DATA_SEGMENTATION_MERGED, "masks.csv"), index=False)